# Basic Plots of Single Pol Radar Data

- Reading and plotting radar data with `pyart`
    - São Roque (SR) (S Band, Single Pol) - variables *corrected_reflectivity*, *corrected_velocity*
    - Cases:
        - 2016-12-25
        - 2017-01-31
        - 2017-03-14
        - 2017-11-15
        - 2017-11-16

## Loading necessary packages

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import glob
from datetime import datetime

import pyart

%matplotlib inline

## Defining necessary functions

### `read_radar_data`
Using radar filenames, read the data and adjust some variables

In [ ]:
def read_radar_data(filename):
    file = pyart.aux_io.read_gamic(filename)
    file_date = pd.to_datetime(file.time['units'][14:])
    
    file.fields['corrected_reflectivity']['standard_name'] = 'corrected_reflectivity'
    file.fields['corrected_velocity']['standard_name'] = 'corrected_velocity'
    
    print('File ' + filename + ' read!')
    return file

### `plot_horizontal_panel`
Using radar and hailpads data, plot horizontal view of fields with hailpad position

In [ ]:
def plot_horizontal_panel(file, hailpads, display_limits, shapefile, radar, save_to, sweep=1):
    #-- Defining date, sweep and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    file_sweep = file.fixed_angle['data'][sweep]
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
    
    #-- Start to plot
    display = pyart.graph.RadarMapDisplay(file)
    fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[9,2.75])
    fig.set_facecolor('w')
    xlim, ylim = display_limits
    
    #--- Reflectivity
    display.plot_ppi_map('corrected_reflectivity', sweep=sweep, ax=ax[0], shapefile=shapefile,
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 10, vmax = 70, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[0], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- Doppler Velocity
    display.plot_ppi_map('corrected_velocity', sweep=sweep, ax=ax[1], shapefile=shapefile,
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = -15, vmax = 15, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[0], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- Common aspects
    plt.suptitle(radar.upper() + ' ' + str(round(file_sweep, 1)) + ' deg ' + str(file_date) + ' UTC', 
                 weight='bold', stretch='condensed', size='xx-large', y=1)
    
    #-- Saving the figure
    plt.savefig(save_to + radar + '_ppi_h_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                dpi=300, transparent=True, bbox_inches='tight')
    plt.close()
    
    return 'Plotting horizontal view for date ' + str(file_date)

### `plot_vertical_panel`
Using radar and hailpads data, plot vertical view of fields with hailpad position

In [ ]:
def plot_vertical_panel(file, hailpads, x_limits, radar, save_to):
    #-- Defining date and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
    
    #-- Plotting only when hailfall occurred
    if file_date.strftime('%H-%M') == hailpad['time_sr']:
        
        #--- Selecting cross-section
        file_cs = pyart.util.cross_section_ppi(file, [hailpad['azim_sr']])
        
        #--- Start to plot
        display = pyart.graph.RadarDisplay(file_cs)
        fig, ax = plt.subplots(1, 2, sharex=True, sharey=True, figsize=[6.5,2])
        fig.set_facecolor('w')
   
        #---- Reflectivity
        display.plot('corrected_reflectivity', vmin=10, vmax=70, title='', ax=ax[0], axislabels_flag=False)
        pnt = ax[0].scatter(x=hailpad['pos_sr'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[0].grid(linestyle=':', c='k')
        
        #---- Doppler Velocity
        display.plot('corrected_velocity', vmin=-15, vmax=15, title='', ax=ax[1], axislabels_flag=False)
        pnt = ax[1].scatter(x=hailpad['pos_sr'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[1].grid(linestyle=':', c='k')
        
        #--- Common aspects
        plt.suptitle(radar.upper() + ' ' + str(file_date) + ' UTC - Azimuth = ' + str(hailpad['azim_sr']) + ' deg', 
                     weight='bold', stretch='condensed', size='x-large', y=1.05)
        plt.xlim(x_limits)
        plt.ylim((0,18))
        
        #--- Manual labelling (outer_label doesn't work!)
        ax[0].set(xlabel='Distance from Radar (km)', ylabel='Distance above\nRadar (km)')
        ax[1].set(xlabel='Distance from Radar (km)', ylabel='')
        
        #--- Saving the figure
        plt.savefig(save_to + radar + '_ppi_v_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                    dpi=300, transparent=True, bbox_inches='tight')
        plt.close()
    
    return 'Plotting vertical view for date ' + str(file_date)

In [ ]:
sr_filenames = open("files_sr_level0.txt").read().split('\n')
hailpad_data = pd.read_csv("../Data/GENERAL/hailpads_registry.txt", index_col=0)
shapefile = "../Data/GENERAL/shapefiles/sao_paulo"
save_path = "figures/ppis/classification/"
radar_name = "sr"

#-- Limits in horizontal view panel
#--- [[xlim], [ylim]]
limits = [[-47.5,-46.5], [-23.3,-22.5]]

#-- Limits (x axis) in vertical panel
#--- (xlim)
xlimits = (50,110)

print(hailpad_data)

In [ ]:
for filename in sr_filenames:
    radar = read_radar_data(filename)
    print(plot_horizontal_panel(radar, hailpad_data, limits, shapefile, radar_name, save_path))
    print(plot_vertical_panel(radar, hailpad_data, xlimits, radar_name, save_path))